In [1]:
# packages
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from torchvision import models, transforms
from torchmetrics.detection import IntersectionOverUnion
from PIL import Image
import os
import cv2
import numpy as np
import json
import shutil

In [2]:
# Check for GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [3]:
#Data base addresses
base_adress = '/kaggle/input/dataset-face-detection-for-edge-computing-class/Dataset_FDDB/Dataset_FDDB/images'
labels_adr = '/kaggle/input/dataset-face-detection-for-edge-computing-class/Dataset_FDDB/Dataset_FDDB/label.txt'

In [4]:
# Make the labels ready

with open(labels_adr, 'r') as f:
    lines = f.readlines()
annotations = []
bboxes = []
flag = False
for line in lines:
    if line.startswith('#'):
      if flag:
        annotations.append({'image':img_name, 'bboxes': bboxes})
        bboxes = []
      flag = True
      img_name = line[2:]
    else:
      x_min, y_min, x_max, y_max = line.split()
      bboxes.append([int(x_min), int(y_min), int(x_max), int(y_max)])

In [5]:
# Custom Dataset Class for FDDB
class FDDBDataset(Dataset):
    def __init__(self, img_dir, annot_file, target_size=(224, 224), transform=None):
        self.img_dir = img_dir
        self.target_size = target_size
        self.transform = transform
        self.data = self._parse_annotations(annot_file)

    def _parse_annotations(self, annot_file):
        
        data = []
        for el in annot_file:
          img_path = os.path.join(self.img_dir, el['image'][:-1])
          boxes = el['bboxes']
          data.append((img_path, boxes))
        return data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        img_path, boxes = self.data[idx]
        image = cv2.imread(img_path)
        if image is None:
            raise FileNotFoundError(f"Image not found: {img_path}")

        # Original dimensions
        h, w, _ = image.shape

        # Resize image
        image_resized = cv2.resize(image, self.target_size)
        target_h, target_w = self.target_size

        # Scale bounding boxes
        scale_x = target_w / w
        scale_y = target_h / h
        boxes_resized = []
        for box in boxes:
            x_min = int(box[0] * scale_x)
            y_min = int(box[1] * scale_y)
            x_max = int(box[2] * scale_x)
            y_max = int(box[3] * scale_y)
            boxes_resized.append([x_min, y_min, x_max, y_max])

        # Convert to tensor
        if self.transform:
            image_resized = self.transform(image_resized)
        else:
            image_resized = transforms.ToTensor()(image_resized)

        return image_resized, torch.tensor(boxes_resized, dtype=torch.float32)

In [6]:
# DataLoader preparation
def get_dataloaders(img_dir, annot_file, batch_size=16, target_size=(224, 224), validation_split=0.2):

    # Transformations
    transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])

    # Dataset
    dataset = FDDBDataset(img_dir, annot_file, target_size, transform)

    # Split dataset
    val_size = int(len(dataset) * validation_split)
    train_size = len(dataset) - val_size
    train_dataset, val_dataset = torch.utils.data.random_split(dataset, [train_size, val_size])

    # DataLoaders
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, collate_fn=collate_fn)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, collate_fn=collate_fn)

    return train_loader, val_loader

def collate_fn(batch):
    """
    Custom collate function to handle variable-length bounding box arrays.

    :param batch: List of tuples (image, boxes).
    :return: Tuple of images and targets.
    """
    images = torch.stack([item[0] for item in batch])
    targets = [item[1] for item in batch]
    return images, targets

># Network Architecture
> **You need to change this architecture**
># NOTE:
> **You are not allowed to use pre-trained models**

In [7]:
from torchsummary import summary

class MyFaceDetector(nn.Module):
    def __init__(self, pretrained=False):
        super().__init__()
        # TODO: resize input images to 448p (or stay 244p?)
        # TODO: threshold the resulting detections by the model's confidence
        # TODO: feature extractor backbone
        # architecture from original YOLO paper

        # conv layer 1
        self.conv1 = nn.Conv2d(
            in_channels=3,
            out_channels=64,
            kernel_size=7,
            stride=2,
            padding=1
        )
        self.relu1 = nn.LeakyReLU(negative_slope=0.1)
        self.maxpool1 = nn.MaxPool2d(kernel_size=2, stride=2)

        self.conv2 = nn.Conv2d(
            in_channels=64,
            out_channels=192,
            kernel_size=3,
            stride=1,
            padding=1
        )
        self.relu2 = nn.LeakyReLU(negative_slope=0.1)
        self.maxpool2 = nn.MaxPool2d(kernel_size=2, stride=2)

        self.conv3 = nn.Conv2d(
            in_channels=192,
            out_channels=128,
            kernel_size=1,
            stride=1,
            padding=1
        )
        self.relu3 = nn.LeakyReLU(negative_slope=0.1)
        self.conv4 = nn.Conv2d(
            in_channels=128,
            out_channels=256,
            kernel_size=3,
            stride=1,
            padding=1
        )
        self.relu4 = nn.LeakyReLU(negative_slope=0.1)
        self.conv5 = nn.Conv2d(
            in_channels=256,
            out_channels=256,
            kernel_size=1,
            stride=1,
            padding=1
        )
        self.relu5 = nn.LeakyReLU(negative_slope=0.1)
        self.conv6 = nn.Conv2d(
            in_channels=256,
            out_channels=512,
            kernel_size=3,
            stride=1,
            padding=1
        )
        self.relu6 = nn.LeakyReLU(negative_slope=0.1)
        self.maxpool3 = nn.MaxPool2d(kernel_size=2, stride=2)

        self.conv7 = nn.Conv2d(
            in_channels=512,
            out_channels=256,
            kernel_size=1,
            stride=1,
            padding=1
        )
        self.relu7 = nn.LeakyReLU(negative_slope=0.1)
        self.conv8 = nn.Conv2d(
            in_channels=256,
            out_channels=512,
            kernel_size=3,
            stride=1,
            padding=1
        )
        self.relu8 = nn.LeakyReLU(negative_slope=0.1)
        self.conv9 = nn.Conv2d(
            in_channels=512,
            out_channels=256,
            kernel_size=1,
            stride=1,
            padding=1
        )
        self.relu9 = nn.LeakyReLU(negative_slope=0.1)
        self.conv10 = nn.Conv2d(
            in_channels=256,
            out_channels=512,
            kernel_size=3,
            stride=1,
            padding=1
        )
        self.relu10 = nn.LeakyReLU(negative_slope=0.1)
        self.conv11 = nn.Conv2d(
            in_channels=512,
            out_channels=256,
            kernel_size=1,
            stride=1,
            padding=1
        )
        self.relu11 = nn.LeakyReLU(negative_slope=0.1)
        self.conv12 = nn.Conv2d(
            in_channels=256,
            out_channels=512,
            kernel_size=3,
            stride=1,
            padding=1
        )
        self.relu12 = nn.LeakyReLU(negative_slope=0.1)
        self.conv13 = nn.Conv2d(
            in_channels=512,
            out_channels=256,
            kernel_size=1,
            stride=1,
            padding=1
        )
        self.relu13 = nn.LeakyReLU(negative_slope=0.1)
        self.conv14 = nn.Conv2d(
            in_channels=256,
            out_channels=512,
            kernel_size=3,
            stride=1,
            padding=1
        )
        self.relu14 = nn.LeakyReLU(negative_slope=0.1)
        self.conv15 = nn.Conv2d(
            in_channels=512,
            out_channels=512,
            kernel_size=1,
            stride=1,
            padding=1
        )
        self.relu15 = nn.LeakyReLU(negative_slope=0.1)
        self.conv16 = nn.Conv2d(
            in_channels=512,
            out_channels=1024,
            kernel_size=3,
            stride=1,
            padding=1
        )
        self.relu16 = nn.LeakyReLU(negative_slope=0.1)
        self.maxpool4 = nn.MaxPool2d(kernel_size=2, stride=2)
        
        self.conv17 = nn.Conv2d(
            in_channels=1024,
            out_channels=512,
            kernel_size=1,
            stride=1,
            padding=1
        )
        self.relu17 = nn.LeakyReLU(negative_slope=0.1)
        self.conv18 = nn.Conv2d(
            in_channels=512,
            out_channels=1024,
            kernel_size=3,
            stride=1,
            padding=1
        )
        self.relu18 = nn.LeakyReLU(negative_slope=0.1)
        # TODO: is the order of multipliers from the architecture correct?
        self.conv19 = nn.Conv2d(
            in_channels=1024,
            out_channels=512,
            kernel_size=1,
            stride=1,
            padding=1
        )
        self.relu19 = nn.LeakyReLU(negative_slope=0.1)
        self.conv20 = nn.Conv2d(
            in_channels=512,
            out_channels=1024,
            kernel_size=3,
            stride=1,
            padding=1
        )
        self.relu20 = nn.LeakyReLU(negative_slope=0.1)
        self.conv21 = nn.Conv2d(
            in_channels=1024,
            out_channels=1024,
            kernel_size=3,
            stride=1,
            padding=1
        )
        self.relu21 = nn.LeakyReLU(negative_slope=0.1)
        self.conv22 = nn.Conv2d(
            in_channels=1024,
            out_channels=1024,
            kernel_size=3,
            stride=2,
            padding=1
        )
        self.relu22 = nn.LeakyReLU(negative_slope=0.1)
        self.conv23 = nn.Conv2d(
            in_channels=1024,
            out_channels=1024,
            kernel_size=3,
            stride=1,
            padding=1
        )
        self.relu23 = nn.LeakyReLU(negative_slope=0.1)
        self.conv24 = nn.Conv2d(
            in_channels=1024,
            out_channels=1024,
            kernel_size=3,
            stride=1,
            padding=1
        )
        self.relu24 = nn.LeakyReLU(negative_slope=0.1)

        self.flatten = nn.Flatten()
        self.linear = nn.Linear(4096, 1)
        
        # (w-k+2p)/s+1
        #linear_layer_input_size = (1024-3+2)/1+1
        #print(linear_layer_input_size)
        
        # regression head
        self.bbox = nn.Sequential(
            nn.Linear(1024*8*8, 4096),
            # TODO: dropout layer as mentioned in paper
            nn.LeakyReLU(negative_slope=0.1),
            nn.Linear(4096, 4)
        )
        
        # classification head
        self.classify = nn.Sequential(
            nn.Linear(1024*8*8, 4096),
            # TODO: dropout layer as mentioned in paper
            nn.LeakyReLU(negative_slope=0.1),
            nn.Linear(4096, 1),
            nn.Sigmoid()
        )

    def forward(self, x):
        # pass through feature extractor backbone
        x = self.conv1(x)
        x = self.relu1(x)
        x = self.maxpool1(x)
        x = self.conv2(x)
        x = self.relu2(x)
        x = self.maxpool2(x)
        x = self.conv3(x)
        x = self.relu3(x)
        x = self.conv4(x)
        x = self.relu4(x)
        x = self.conv5(x)
        x = self.relu5(x)
        x = self.conv6(x)
        x = self.relu6(x)
        x = self.maxpool3(x)
        x = self.conv7(x)
        x = self.relu7(x)
        x = self.conv8(x)
        x = self.relu8(x)
        x = self.conv9(x)
        x = self.relu9(x)
        x = self.conv10(x)
        x = self.relu10(x)
        x = self.conv11(x)
        x = self.relu11(x)
        x = self.conv12(x)
        x = self.relu12(x)
        x = self.conv13(x)
        x = self.relu13(x)
        x = self.conv14(x)
        x = self.relu14(x)
        x = self.conv15(x)
        x = self.relu15(x)
        x = self.conv16(x)
        x = self.relu16(x)
        x = self.maxpool4(x)
        x = self.conv17(x)
        x = self.relu17(x)
        x = self.conv18(x)
        x = self.relu18(x)
        x = self.conv19(x)
        x = self.relu19(x)
        x = self.conv20(x)
        x = self.relu20(x)
        x = self.conv21(x)
        x = self.relu21(x)
        x = self.conv22(x)
        x = self.relu22(x)
        x = self.conv23(x)
        x = self.relu23(x)
        x = self.conv24(x)
        x = self.relu24(x)

        #_, c, h, w = x.shape
        #print(c, h, w)
        
        x = self.flatten(x)
        #x = torch.flatten(x, 1)
        #x = self.linear(x)
        
        bbox = self.bbox(x)
        label = self.classify(x)

        return bbox, label
model = MyFaceDetector().to(device)
summary(model, (3, 224, 224))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 110, 110]           9,472
         LeakyReLU-2         [-1, 64, 110, 110]               0
         MaxPool2d-3           [-1, 64, 55, 55]               0
            Conv2d-4          [-1, 192, 55, 55]         110,784
         LeakyReLU-5          [-1, 192, 55, 55]               0
         MaxPool2d-6          [-1, 192, 27, 27]               0
            Conv2d-7          [-1, 128, 29, 29]          24,704
         LeakyReLU-8          [-1, 128, 29, 29]               0
            Conv2d-9          [-1, 256, 29, 29]         295,168
        LeakyReLU-10          [-1, 256, 29, 29]               0
           Conv2d-11          [-1, 256, 31, 31]          65,792
        LeakyReLU-12          [-1, 256, 31, 31]               0
           Conv2d-13          [-1, 512, 31, 31]       1,180,160
        LeakyReLU-14          [-1, 512,

In [8]:
class MobileNetV2FaceDetector(nn.Module):
    def __init__(self, pretrained=True):
        super(MobileNetV2FaceDetector, self).__init__()
        # Load MobileNetV2 base
        self.base = models.mobilenet_v2(pretrained=pretrained).features
        self.pool = nn.AdaptiveAvgPool2d(1)

        # Custom head for bounding box and classification
        self.fc_bbox = nn.Sequential(
            nn.Linear(1280, 512),
            nn.ReLU(),
            nn.Linear(512, 4),  # Bounding box: [x_min, y_min, x_max, y_max]
        )
        self.fc_label = nn.Sequential(
            nn.Linear(1280, 512),
            nn.ReLU(),
            nn.Linear(512, 1),  # Binary classification: face/no face
            nn.Sigmoid(),
        )

    def forward(self, x):
        x = self.base(x)
        x = self.pool(x).view(x.size(0), -1)
        bbox = self.fc_bbox(x)
        label = self.fc_label(x)
        return bbox, label

#model = MobileNetV2FaceDetector().to(device)
# after 100 epochs, IoU was ~0.56

In [9]:
# loss / optimizer / train
# Loss functions
bbox_loss_fn = nn.SmoothL1Loss()  # For bounding box regression
label_loss_fn = nn.BCELoss()      # For binary classification

# Optimizer
optimizer = optim.Adam(model.parameters(), lr=1e-3)

# Training loop
def train_model(model, train_loader, val_loader, num_epochs=10):
    best_val_loss = float('inf')  # Initialize best validation loss
    best_model_path = "best_model.pth"  # Path to save the best model
    for epoch in range(num_epochs):
        model.train()
        total_loss = 0
        for images, targets in train_loader:
            images = images.to(device)
            bboxes = [torch.tensor(t, dtype=torch.float32).to(device) for t in targets]  # List of bounding boxes
            labels = [int(1) for t in targets]  # List of labels
            labels = torch.tensor(labels, dtype=torch.float32).to(device)
            preds_bbox, preds_label = model(images)
            # Compute losses
            bbox_losses = []
            label_losses = []
            for i in range(len(bboxes)):
              bbox_losses.append(bbox_loss_fn(preds_bbox[i], bboxes[i]))
              label_losses.append(label_loss_fn(preds_label[i], labels[i].unsqueeze(-1)))

            bbox_loss = torch.mean(torch.stack(bbox_losses))
            label_loss = torch.mean(torch.stack(label_losses))
            loss = bbox_loss + label_loss

            # Backward pass and optimization
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            total_loss += loss.item()

        print(f"Epoch {epoch+1}/{num_epochs}, Loss: {total_loss / len(train_loader):.4f}")

        # Validate and save the best model
        val_loss = validate_model(model, val_loader)
        if val_loss < best_val_loss:
            print(f"Validation loss improved from {best_val_loss:.4f} to {val_loss:.4f}. Saving model...")
            best_val_loss = val_loss
            torch.save(model.state_dict(), best_model_path)

    print("Training complete. Best model saved as:", best_model_path)


In [10]:
def normalize_boxes(preds):
    """
    Normalize the 'boxes' in the predictions to ensure they are all tensors of shape [N, 4].
    Args:
        preds: List of dictionaries with 'boxes' and 'labels'.
    Returns:
        Normalized predictions with 'boxes' as tensors of shape [N, 4].
    """
    for pred in preds:
        # If boxes is a list of tensors, stack them into a single tensor
        if isinstance(pred['boxes'], list):
            pred['boxes'] = torch.stack(pred['boxes'])  # Stack into [N, 4]
    return preds

In [11]:
def validate_model(model, val_loader):
    metric = IntersectionOverUnion().to(device)
    model.eval()
    total_bbox_loss = 0
    total_label_loss = 0
    total_iou = []
    with torch.no_grad():
        for images, targets in val_loader:
            images = images.to(device)
            bboxes = [torch.tensor(t, dtype=torch.float32).to(device) for t in targets]  # List of bounding boxes
            labels = [int(1) for t in targets]  # List of labels
            labels = torch.tensor(labels, dtype=torch.float32).to(device)
            preds_bbox, preds_label = model(images)
            # print('labels')
            # print(preds_label)
            # input()
            bbox_losses = []
            label_losses = []
            # print(bboxes)
            # print('//////////////////////////////')
            for i in range(len(bboxes)):
              bbox_losses.append(bbox_loss_fn(preds_bbox[i], bboxes[i]))
              label_losses.append(label_loss_fn(preds_label[i], labels[i].unsqueeze(-1)))
            #   print([bboxes[i]])
            #   print('///////////////////////////////////////')
            #   print(preds_bbox[i].shape)
              preds = [
                {"boxes": [preds_bbox[i]], "labels": preds_label[i]}
                ]
              preds = normalize_boxes(preds)
            #   print("Preds")
            #   print(preds)
              targets_combined = torch.cat([bboxes[i]], dim=0)
                # print(targets_combined)
              targets = [
                {"boxes": targets_combined, "labels": torch.ones(len(targets_combined)).to(device)}
                ]
              iou_value = metric(preds, targets)
              total_iou.append(iou_value['iou'].item())
            #   targets = targets.to(device)
            #   print("Targets")
            #   print(targets)

            total_bbox_loss += torch.mean(torch.stack(bbox_losses))
            total_label_loss += torch.mean(torch.stack(label_losses))
            # print(targets)
            # loss = total_label_loss + total_label_loss

            # total_bbox_loss += bbox_loss_fn(preds_bbox, bboxes).item()
            # total_label_loss += label_loss_fn(preds_label, labels).item()

    # Calculate average validation loss
    avg_bbox_loss = total_bbox_loss / len(val_loader)
    avg_label_loss = total_label_loss / len(val_loader)
    val_loss = avg_bbox_loss + avg_label_loss
    print('IoU = ', sum(total_iou)/len(total_iou))
    print(f"Validation - BBox Loss: {avg_bbox_loss:.4f}, Label Loss: {avg_label_loss:.4f}, Total Loss: {val_loss:.4f}")
    return val_loss

In [12]:
# train
batch_size = 32
target_size = (224, 224)
train_loader, val_loader = get_dataloaders(base_adress, annotations, batch_size, target_size)
train_model(model, train_loader, val_loader, num_epochs=5)

<ipython-input-9-9f95b738bbcf>:18: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  bboxes = [torch.tensor(t, dtype=torch.float32).to(device) for t in targets]  # List of bounding boxes
/usr/local/lib/python3.10/dist-packages/torch/nn/modules/loss.py:1040: UserWarning: Using a target size (torch.Size([1, 4])) that is different to the input size (torch.Size([4])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.smooth_l1_loss(input, target, reduction=self.reduction, beta=self.beta)
/usr/local/lib/python3.10/dist-packages/torch/nn/modules/loss.py:1040: UserWarning: Using a target size (torch.Size([2, 4])) that is different to the input size (torch.Size([4])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.smoot

Epoch 1/5, Loss: 19667034.2841


<ipython-input-11-3ae0e263f413>:10: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  bboxes = [torch.tensor(t, dtype=torch.float32).to(device) for t in targets]  # List of bounding boxes
/usr/local/lib/python3.10/dist-packages/torch/nn/modules/loss.py:1040: UserWarning: Using a target size (torch.Size([12, 4])) that is different to the input size (torch.Size([4])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.smooth_l1_loss(input, target, reduction=self.reduction, beta=self.beta)


IoU =  0.0
Validation - BBox Loss: 4181522.7500, Label Loss: 100.0000, Total Loss: 4181622.7500
Validation loss improved from inf to 4181622.7500. Saving model...
Epoch 2/5, Loss: 29001948557.9366
IoU =  0.0
Validation - BBox Loss: 274537664.0000, Label Loss: 100.0000, Total Loss: 274537760.0000
Epoch 3/5, Loss: 143842620.6338
IoU =  0.0
Validation - BBox Loss: 3808403.2500, Label Loss: 100.0000, Total Loss: 3808503.2500
Validation loss improved from 4181622.7500 to 3808503.2500. Saving model...
Epoch 4/5, Loss: 1813690.9780
IoU =  0.0
Validation - BBox Loss: 924771.8125, Label Loss: 100.0000, Total Loss: 924871.8125
Validation loss improved from 3808503.2500 to 924871.8125. Saving model...
Epoch 5/5, Loss: 761294.7232
IoU =  0.0
Validation - BBox Loss: 557056.6875, Label Loss: 100.0000, Total Loss: 557156.6875
Validation loss improved from 924871.8125 to 557156.6875. Saving model...
Training complete. Best model saved as: best_model.pth


In [13]:
def calculate_iou(pred_box, gt_box):
    """
    Calculate IoU (Intersection over Union) for a single pair of boxes.
    Args:
        pred_box: Tensor of shape (4,), [x_min, y_min, x_max, y_max].
        gt_box: Tensor of shape (4,), [x_min, y_min, x_max, y_max].
    Returns:
        IoU value (float).
    """
    # Determine the (x, y)-coordinates of the intersection rectangle
    x1 = max(pred_box[0], gt_box[0])
    y1 = max(pred_box[1], gt_box[1])
    x2 = min(pred_box[2], gt_box[2])
    y2 = min(pred_box[3], gt_box[3])

    # Compute the area of intersection rectangle
    inter_width = max(0, x2 - x1)
    inter_height = max(0, y2 - y1)
    inter_area = inter_width * inter_height

    # Compute the area of both the predicted and ground-truth rectangles
    pred_area = (pred_box[2] - pred_box[0]) * (pred_box[3] - pred_box[1])
    gt_area = (gt_box[2] - gt_box[0]) * (gt_box[3] - gt_box[1])

    # Compute the area of union
    union_area = pred_area + gt_area - inter_area

    # Compute IoU
    iou = inter_area / union_area if union_area > 0 else 0.0
    return iou

# You can write your function to evaluate your trained model
> You can use calculate_iou function in you evaluation function

In [14]:
def evaluate():
    pass

# This is for the next part of the assignment

# Exporting to ONNX

In [15]:
#!pip install onnx
#!pip install onnxscript

In [16]:
"""
# Model class must be defined somewhere
PATH = '/kaggle/working/best_model.pth'
model = MobileNetV2FaceDetector().to(device)
model.load_state_dict(torch.load(PATH))
model.eval()
dummy_input = torch.randn(1, 3, 224, 224).to(device)  # Adjust shape based on your model's input size

# Export the model to ONNX
torch.onnx.export(
    model,  # The loaded PyTorch model
    dummy_input,  # Example input tensor
    "model.onnx",  # Output ONNX file name
    export_params=True,  # Store trained parameters
    opset_version=13,  # ONNX version (adjust as needed)
    do_constant_folding=True,  # Optimize by folding constants
    input_names=["input"],  # Naming input tensor
    output_names=["output"],  # Naming output tensor
    dynamic_axes=None 
)

print("Model successfully exported to ONNX!")
"""

'\n# Model class must be defined somewhere\nPATH = \'/kaggle/working/best_model.pth\'\nmodel = MobileNetV2FaceDetector().to(device)\nmodel.load_state_dict(torch.load(PATH))\nmodel.eval()\ndummy_input = torch.randn(1, 3, 224, 224).to(device)  # Adjust shape based on your model\'s input size\n\n# Export the model to ONNX\ntorch.onnx.export(\n    model,  # The loaded PyTorch model\n    dummy_input,  # Example input tensor\n    "model.onnx",  # Output ONNX file name\n    export_params=True,  # Store trained parameters\n    opset_version=13,  # ONNX version (adjust as needed)\n    do_constant_folding=True,  # Optimize by folding constants\n    input_names=["input"],  # Naming input tensor\n    output_names=["output"],  # Naming output tensor\n    dynamic_axes=None \n)\n\nprint("Model successfully exported to ONNX!")\n'